# 안면 Multi-Label (성별/나이) Classification

In [14]:
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

### 데이터프레임 로드

In [15]:
df=pd.read_csv('data/FamilyData/Val/FamilyData_Val_DF.csv',index_col=0)
df.sample(20)

,img_path,M,F,0's,10's,20's,30's,40's,50's,60+
3173,F0900_IND_D_34_-90_02.JPG,0,1,0,0,0,1,0,0,0
1058,F0833_IND_S_15_90_01.JPG,1,0,0,1,0,0,0,0,0
2810,F0888_IND_D3_8_-45_01.JPG,0,1,1,0,0,0,0,0,0
893,F0828_IND_M_41_-90_02.JPG,0,1,0,0,0,0,1,0,0
139,F0805_IND_F_46_90_02.JPG,1,0,0,0,0,0,1,0,0
2585,F0881_IND_D_20_0_02.JPG,0,1,0,0,1,0,0,0,0
755,F0824_IND_F_49_0_02.JPG,1,0,0,0,0,0,1,0,0
1885,F0860_IND_F_43_0_02.JPG,1,0,0,0,0,0,1,0,0
2577,F0880_IND_S_11_45_02.JPG,1,0,0,1,0,0,0,0,0
1420,F0845_IND_D_31_-45_01.jpg,0,1,0,0,0,1,0,0,0


In [16]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, random_state=777, test_size=0.2)
train_df, val_df = train_test_split(train_df, random_state=777, test_size=0.2)

len(train_df) , len(val_df), len(test_df)

(2048, 512, 640)

In [17]:
train_df[:5]

,img_path,M,F,0's,10's,20's,30's,40's,50's,60+
2371,F0874_IND_S_33_-45_02.JPG,1,0,0,0,0,1,0,0,0
1432,F0845_IND_F_60_-90_01.jpg,1,0,0,0,0,0,0,0,1
356,F0812_IND_S2_13_45_01.JPG,1,0,0,1,0,0,0,0,0
1501,F0847_IND_D_30_-45_02.jpg,0,1,0,0,0,1,0,0,0
22,F0801_IND_S_15_-90_01.JPG,1,0,0,1,0,0,0,0,0


In [18]:
val_df[:5]

,img_path,M,F,0's,10's,20's,30's,40's,50's,60+
3017,F0894_IND_M_45_45_02.JPG,0,1,0,0,0,0,1,0,0
1039,F0833_IND_GM_69_90_02.JPG,0,1,0,0,0,0,0,0,1
2966,F0893_IND_D2_16_45_01.JPG,0,1,0,1,0,0,0,0,0
45,F0802_IND_M_44_0_02.JPG,0,1,0,0,0,0,1,0,0
527,F0816_IND_S_20_45_02.JPG,1,0,0,0,1,0,0,0,0


In [19]:
test_df[:5]

,img_path,M,F,0's,10's,20's,30's,40's,50's,60+
186,F0807_IND_D_9_45_01.JPG,0,1,1,0,0,0,0,0,0
621,F0820_IND_M_54_-45_02.JPG,0,1,0,0,0,0,0,1,0
980,F0831_IND_S_13_-45_01.JPG,1,0,0,1,0,0,0,0,0
1648,F0852_IND_F_44_90_01.jpg,1,0,0,0,0,0,1,0,0
3193,F0900_IND_M_57_-90_02.JPG,0,1,0,0,0,0,0,1,0


In [20]:
columns= list(df.columns)[1:]
columns

['M', 'F', "0's", "10's", "20's", "30's", "40's", "50's", '60+']

### ImageGenerator 정의

In [21]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [22]:
preprocessed_data_path = 'E:/045.가족 관계가 알려진 얼굴 이미지 데이터/01.데이터/Preprocessed_Datasets'

train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=preprocessed_data_path,
    x_col="img_path",
    y_col=columns,
    batch_size=32,
    seed=777,
    shuffle=True,
    class_mode="raw",
    target_size=(224,224))

Found 2048 validated image filenames.


In [23]:
valid_generator=test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=preprocessed_data_path,
    x_col="img_path",
    y_col=columns,
    batch_size=32,
    seed=777,
    shuffle=True,
    class_mode="raw",
    target_size=(224,224))

Found 512 validated image filenames.


In [24]:
# test_generator은 batch_size 1개씩
test_generator=test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=preprocessed_data_path,
    x_col="img_path",
    batch_size=1,
    seed=777,
    shuffle=False,
    class_mode=None,
    target_size=(224,224))

Found 640 validated image filenames.


In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras import Model

inp = Input(shape = (224,224,3))

x = Conv2D(32, (3, 3), padding = 'same')(inp)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), padding = 'same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
output1= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output2= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output3= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output4= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output5= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output6= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output7= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output8= tf.keras.layers.Dense(1, activation='sigmoid')(x)
output9= tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = Model(inp,[output1,output2,output3,output4,output5,output6,output7,output8,output9])
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, rho=0.9),
              loss=['binary_crossentropy', 'binary_crossentropy','binary_crossentropy','binary_crossentropy',
                    'binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy'],
              metrics=['accuracy']),

(None,)

In [26]:
from PIL import Image
# import PIL

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
hist = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

C:\Users\user\AppData\Local\Temp\ipykernel_22960\2602555156.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(generator=train_generator,


Epoch 1/5
64/64 [==============================] - 409s 6s/step - loss: 5.2442 - dense_11_loss: 0.5795 - dense_12_loss: 0.5853 - dense_13_loss: 0.5815 - dense_14_loss: 0.5764 - dense_15_loss: 0.5915 - dense_16_loss: 0.5893 - dense_17_loss: 0.5735 - dense_18_loss: 0.5872 - dense_19_loss: 0.5801 - dense_11_accuracy: 0.7726 - dense_12_accuracy: 0.7667 - dense_13_accuracy: 0.7667 - dense_14_accuracy: 0.7696 - dense_15_accuracy: 0.7721 - dense_16_accuracy: 0.7721 - dense_17_accuracy: 0.7729 - dense_18_accuracy: 0.7705 - dense_19_accuracy: 0.7705 - val_loss: 5.0438 - val_dense_11_loss: 0.5576 - val_dense_12_loss: 0.5596 - val_dense_13_loss: 0.5598 - val_dense_14_loss: 0.5598 - val_dense_15_loss: 0.5608 - val_dense_16_loss: 0.5582 - val_dense_17_loss: 0.5609 - val_dense_18_loss: 0.5634 - val_dense_19_loss: 0.5637 - val_dense_11_accuracy: 0.7778 - val_dense_12_accuracy: 0.7778 - val_dense_13_accuracy: 0.7778 - val_dense_14_accuracy: 0.7778 - val_dense_15_accuracy: 0.7778 - val_dense_16_accurac

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()